# 2.5k Asset Backtest Example - SPO

## 1. Setup

#### 1.1 Load InvestOS module

In [1]:
# Add relative path to module lookup path...
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

# ... then import module
import investos as inv

# Import other required modules
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.plotting.backend = "plotly"
import numpy as np

#### 1.2 Load historical stock prices and volumes

Place in examples/data directory [data from Kaggle](https://www.kaggle.com/datasets/camnugent/sandp500/download?datasetVersionNumber=4)

Note: you will need to create (and sign into) a free Kaggle account

In [2]:
# A. Load S&P 500 tickers, names, and industries
dir_name = './data/'

df = pd.read_parquet(
    dir_name + '10k_synthetic.parquet'
)

#### 1.3 Split dfs into historical and forecast

In [3]:
import datetime

date_split_forecasts = datetime.date(2022, 1, 1)

df_forecast = df[df.date >= date_split_forecasts][['asset', 'date', 'return']]
df_actual = df

#### 1.4 Create (fake) forecasts

In [4]:
np.random.seed(0)

# Median (daily) return is VERY close to 0:
print("Median return:", df_forecast['return'].median())

# Cap returns at +- 10%...
df_forecast['return'] = df_forecast['return'].clip(-0.1, 0.1)

# ... then reduce signal...
df_forecast['return'] /= 10

# ... then add guassian noise to daily returns
std = df_actual[df_actual.date >= date_split_forecasts]['return'].var() ** 0.5
noise = np.random.normal(0, std, size=df_forecast.shape[0])

df_forecast['return'] = df_forecast['return'] + noise

Median return: 0.0002618778449523214


In [5]:
# Make sure predictions aren't too accurate:

agree_on_sign = np.sign(df_forecast['return']) == np.sign(
    df_actual[df_actual.date >= date_split_forecasts]['return']
)

print(
    "Return predictions have the right sign %.1f%% of the time" %
    ((agree_on_sign.sum() / agree_on_sign.shape[0]) * 100)
)

Return predictions have the right sign 53.2% of the time


## 2. Portfolio optimization

#### 2.1 Create portfolio optimization instance

In [6]:
from investos.portfolio.cost_model import *
from investos.portfolio.constraint_model import *
from investos.portfolio.risk_model import *

In [7]:
strategy = inv.portfolio.strategy.SPO(risk_model = None)

portfolio = inv.portfolio.Controller(
    df_forecast,
    df_actual,
    strategy=strategy,
    after_step=[
        lambda backtest, t, u, h_next: print("Optimizing for month", f"{t.year}-{t.month}") if t.day == 1 else None,
        lambda backtest, t, u, h_next: print(t),

    ]
)

In [8]:
backtest_result = portfolio.generate_positions()

Optimizing...


/opt/homebrew/lib/python3.9/site-packages/cvxpy/problems/problem.py:161: UserWarning: Constraint #3 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/opt/homebrew/lib/python3.9/site-packages/cvxpy/problems/problem.py:161: UserWarning: Constraint #11 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "


2022-01-03


KeyboardInterrupt: 

#### 2.2 Backtest results

In [ ]:
fig = backtest_result.v.plot()
fig.layout.update(showlegend=False)
fig.show()

In [ ]:
backtest_result.summary